In [203]:
import pandas as pd
import numpy as np

from scipy.sparse import csr_matrix

from sklearn.neighbors import NearestNeighbors

In [268]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

movies.head(20)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [205]:
ratings.head(10)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
5,1,70,3.0,964982400
6,1,101,5.0,964980868
7,1,110,4.0,964982176
8,1,151,5.0,964984041
9,1,157,5.0,964984100


In [206]:
movies.drop(['genres'], axis = 1, inplace = True)
movies.head(10)

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)
5,6,Heat (1995)
6,7,Sabrina (1995)
7,8,Tom and Huck (1995)
8,9,Sudden Death (1995)
9,10,GoldenEye (1995)


In [207]:
ratings.drop(['timestamp'], axis = 1, inplace = True)
ratings.head(10)

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
5,1,70,3.0
6,1,101,5.0
7,1,110,4.0
8,1,151,5.0
9,1,157,5.0


In [208]:
user_item_matrix = ratings.pivot(index = 'movieId', columns = 'userId', values= 'rating')
user_item_matrix.head(10)
print(user_item_matrix.shape)

(9724, 610)


In [209]:
user_item_matrix.fillna(0, inplace = True)
user_item_matrix.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,0.0,4.0,0.0,4.5,0.0,0.0,0.0,...,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
2,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,...,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0


In [210]:
user_item_matrix.shape

(9724, 610)

In [211]:
users_votes = ratings.groupby('userId')['rating'].agg('count')

movies_votes = ratings.groupby('movieId')['rating'].agg('count')

In [212]:
user_mask = users_votes[users_votes > 50].index
movie_mask = movies_votes[movies_votes > 10].index

In [213]:
user_item_matrix = user_item_matrix.loc[movie_mask,:]

user_item_matrix = user_item_matrix.loc[:,user_mask]

In [214]:
user_item_matrix.shape

(2121, 378)

In [215]:
csr_data = csr_matrix(user_item_matrix.values)

print(csr_data[:2,:5])

  (0, 0)	4.0
  (0, 3)	4.5
  (1, 2)	4.0


In [216]:
user_item_matrix = user_item_matrix.rename_axis(None, axis = 1).reset_index()
user_item_matrix.head()

,movieId,1,4,6,7,10,11,15,16,17,...,600,601,602,603,604,605,606,607,608,610
0,1,4.0,0.0,0.0,4.5,0.0,0.0,2.5,0.0,4.5,...,2.5,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,5.0
1,2,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0
2,3,4.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
3,5,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.5,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0
4,6,4.0,0.0,4.0,0.0,0.0,5.0,0.0,0.0,0.0,...,0.0,0.0,3.0,4.0,3.0,0.0,0.0,0.0,0.0,5.0


In [217]:
knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute', n_neighbors = 20, n_jobs = -1)

knn.fit(csr_data)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=20)

In [280]:
recommendations = 10

search_word = 'Shrek'

In [281]:
movie_search = movies[movies['title'].str.contains(search_word)]
movie_search

,movieId,title,genres
3194,4306,Shrek (2001),Adventure|Animation|Children|Comedy|Fantasy|Ro...
5160,8360,Shrek 2 (2004),Adventure|Animation|Children|Comedy|Musical|Ro...
6486,53121,Shrek the Third (2007),Adventure|Animation|Children|Comedy|Fantasy
6915,64249,Shrek the Halls (2007),Adventure|Animation|Comedy|Fantasy
7360,78637,Shrek Forever After (a.k.a. Shrek: The Final C...,Adventure|Animation|Children|Comedy|Fantasy|IMAX


In [282]:
movie_id = movie_search.iloc[0]['movieId']

movie_id = user_item_matrix[user_item_matrix['movieId'] == movie_id].index[0]
movie_id

1292

In [283]:
distances, indices = knn.kneighbors(csr_data[movie_id], n_neighbors = recommendations + 1)

In [284]:
indices

array([[1292, 1473, 1607, 1347, 1484, 1362, 1569, 1536, 1443, 1350, 1363]])

In [285]:
distances

array([[2.22044605e-16, 2.56021060e-01, 2.71754111e-01, 2.88389805e-01,
        2.94953823e-01, 2.97651022e-01, 3.02120185e-01, 3.21050307e-01,
        3.33704930e-01, 3.42930294e-01, 3.53788350e-01]])

In [286]:
indices_list = indices.squeeze().tolist()
distances_list = distances.squeeze().tolist()

indices_distances = list(zip(indices_list, distances_list))

print(type(indices_distances[0]))

print(indices_distances[:3])

<class 'tuple'>
[(1292, 2.220446049250313e-16), (1473, 0.2560210598196109), (1607, 0.2717541112543478)]


In [287]:
indices_distances_sorted = sorted(indices_distances, key = lambda x: x[1], reverse = False)


indices_distances_sorted = indices_distances_sorted[1:]
indices_distances_sorted

[(1473, 0.2560210598196109),
 (1607, 0.2717541112543478),
 (1347, 0.2883898045694022),
 (1484, 0.29495382273139703),
 (1362, 0.29765102233201457),
 (1569, 0.30212018464549895),
 (1536, 0.32105030667738543),
 (1443, 0.33370492981720723),
 (1350, 0.3429302944784337),
 (1363, 0.3537883502362845)]

In [288]:
recom_list = []


for ind_dist in indices_distances_sorted:

    matrix_movie_id = user_item_matrix.iloc[ind_dist[0]]['movieId']

    id = movies[movies['movieId'] == matrix_movie_id].index

    title = movies.iloc[id]['title'].values[0]
    dist = ind_dist[1]

    recom_list.append({'Title' : title, 'Distance' : dist})

In [289]:
recom_list[0]

{'Title': 'Finding Nemo (2003)', 'Distance': 0.2560210598196109}

In [290]:
recom_df = pd.DataFrame(recom_list, index = range(1, recommendations + 1))
recom_df

,Title,Distance
1,Finding Nemo (2003),0.256021
2,"Incredibles, The (2004)",0.271754
3,"Monsters, Inc. (2001)",0.288390
4,Pirates of the Caribbean: The Curse of the Bla...,0.294954
5,"Lord of the Rings: The Fellowship of the Ring,...",0.297651
6,Shrek 2 (2004),0.302120
7,"Lord of the Rings: The Return of the King, The...",0.321050
8,"Lord of the Rings: The Two Towers, The (2002)",0.333705
9,Harry Potter and the Sorcerer's Stone (a.k.a. ...,0.342930
10,"Beautiful Mind, A (2001)",0.353788
